In [1]:
#-*- coding:utf-8
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution1D, Conv2D, MaxPooling2D, MaxPooling1D, LSTM, Embedding
from keras.optimizers import SGD
import keras.backend as K
import random
import re
import tensorflow as tf
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# config_proto = tf.ConfigProto(log_device_placement=0,allow_soft_placement=0)
# config_proto.gpu_options.allow_growth = True
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

Using TensorFlow backend.


In [2]:
X_train = np.load('../../20_Galaxy/ANGRY/2HXT-train.npy')
Y_train = np.load('../../20_Galaxy/ANGRY/2HXT-train_label.npy')

X_valid = np.load('../../20_Galaxy/ANGRY/2HXT-val.npy')
Y_valid = np.load('../../20_Galaxy/ANGRY/2HXT-val_label.npy')

In [3]:
one_hot = preprocessing.OneHotEncoder(sparse = False)
y_train = one_hot.fit_transform(Y_train)
y_valid = one_hot.fit_transform(Y_valid)

/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [26]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils
def conv_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    x = Activation('relu')(input)
    x = Conv1D(nb_filter, (9), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    return x

def dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    feature_list = [x]

    for i in range(nb_layers):
        x = conv_block(x, growth_rate, dropout_rate, weight_decay)
        feature_list.append(x)
        x = Concatenate(axis=concat_axis)(feature_list)
        nb_filter += growth_rate

    return x, nb_filter

def transition_block(input, nb_filter, dropout_rate=None, weight_decay=1E-4):
    concat_axis = 1 if K.image_dim_ordering() == "th" else -1

    x = Conv1D(nb_filter, (1), kernel_initializer="he_uniform", padding="same", use_bias=False,
                      kernel_regularizer=l2(weight_decay))(input)
    if dropout_rate is not None:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling1D((2), strides=(2))(x)

    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                           beta_regularizer=l2(weight_decay))(x)

    return x

def createDenseNet(nb_classes, img_dim, depth=40, nb_dense_block=3, growth_rate=12, nb_filter=16, dropout_rate=None,
                     weight_decay=1E-4, verbose=True):

    model_input = Input(shape=img_dim)

    for i in range(2):
        model_input0 = model_input[:,i]
        print model_input[:,i]
    
        concat_axis = 1 if K.image_dim_ordering() == "th" else -1

        assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"

        # layers in each dense block
        nb_layers = int((depth - 4) / 3)

        # Initial convolution
        x = Conv1D(nb_filter, (9), kernel_initializer="he_uniform", padding="same", name="initial_conv1D", use_bias=False,
                          kernel_regularizer=l2(weight_decay))(model_input0)

        x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay),
                                beta_regularizer=l2(weight_decay))(x)

        # Add dense blocks
        for block_idx in range(nb_dense_block - 1):
            x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                       weight_decay=weight_decay)
            # add transition_block
            x = transition_block(x, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)

        # The last dense_block does not have a transition_block
        x, nb_filter = dense_block(x, nb_layers, nb_filter, growth_rate, dropout_rate=dropout_rate,
                                   weight_decay=weight_decay)

        x = Activation('relu')(x)
        if i == 0:
            out = x
        else:
            out = Concatenate(axis=concat_axis)([out,x])
        print out.shape
#         x = LSTM(128,return_sequences=True)(x)
    x = Flatten()(x)
#     print x.shape
#         x = GlobalAveragePooling2D()(x)
    x = Dense(nb_classes, activation='softmax', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(out)

    densenet = Model(inputs=model_input, outputs=x)

    if verbose: 
        print("DenseNet-%d-%d created." % (depth, growth_rate))

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)
densenet_depth = 40
densenet_growth_rate = 12

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim,depth=densenet_depth,
                  growth_rate = densenet_growth_rate)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=12, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("strided_slice_108:0", shape=(?, 3640, 1), dtype=float32)
(?, 910, 448)
Tensor("strided_slice_110:0", shape=(?, 3640, 1), dtype=float32)
(?, 910, 1328)


AttributeError: 'NoneType' object has no attribute '_inbound_nodes'

In [35]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def createDenseNet(nb_classes, img_dim):

    model_input = Input(shape=img_dim)

    cnn1 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
    cnn2 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
    pool1 = MaxPooling1D(pool_size=(7))(cnn2)
    drop1 = Dropout(0.25)(pool1)

    cnn3 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
    cnn4 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
    pool2 = MaxPooling1D(pool_size=(6))(cnn4)
    drop2 = Dropout(0.25)(pool2)

    cnn5 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
    cnn6 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
    pool3 = MaxPooling1D(pool_size=(6))(cnn6)
    drop3 = Dropout(0.25)(pool3)
    
    flatten1 = Flatten()(drop3)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)
# densenet_depth = 46
# densenet_growth_rate = 8

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 42s 3ms/step - loss: 0.5908 - acc: 0.6749 - val_loss: 0.5553 - val_acc: 0.8088
Epoch 2/50
15658/15658 [==============================] - 17s 1ms/step - loss: 0.4880 - acc: 0.7686 - val_loss: 0.5640 - val_acc: 0.8605
Epoch 3/50
15658/15658 [==============================] - 18s 1ms/step - loss: 0.4562 - acc: 0.7916 - val_loss: 0.4511 - val_acc: 0.8633
Epoch 4/50
15658/15658 [==============================] - 18s 1ms/step - loss: 0.4427 - acc: 0.8021 - val_loss: 0.4642 - val_acc: 0.8710
Epoch 5/50
15658/15658 [==============================] - 18s 1ms/step - loss: 0.4298 - acc: 0.8064 - val_loss: 0.4687 - val_acc: 0.8390
Epoch 6/50
15658/15658 [==============================] - 18s 1ms/step - loss: 0.4225 - acc: 0.8114 - val_loss: 0.5927 - val_acc: 0.8670
Epoch 7/50
15658/15658 [==============================] - 18s 1ms/step - loss: 0.4099 - acc: 0.8173 - val_loss: 0.4624 - val_acc:

In [67]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def pre(x):
    x = x[:,0]
    return x

def createDenseNet(nb_classes, img_dim):

    model_input = Input(shape=img_dim)
    
#     for i in range(1):
    inputs = Lambda(pre, name='pre')(model_input)
    print inputs
    print model_input
    
    cnn1 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
    cnn2 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
    pool1 = MaxPooling1D(pool_size=(7))(cnn2)
    drop1 = Dropout(0.25)(pool1)

    cnn3 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
    cnn4 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
    pool2 = MaxPooling1D(pool_size=(6))(cnn4)
    drop2 = Dropout(0.25)(pool2)

    cnn5 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
    cnn6 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
    pool3 = MaxPooling1D(pool_size=(6))(cnn6)
    drop3 = Dropout(0.25)(pool3)
    
    flatten1 = Flatten()(drop3)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=25, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("pre_4/strided_slice:0", shape=(?, 3640, 1), dtype=float32)
Tensor("input_62:0", shape=(?, 12, 3640, 1), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 39s 2ms/step - loss: 0.6924 - acc: 0.5236 - val_loss: 0.6939 - val_acc: 0.5015
Epoch 2/50
15658/15658 [==============================] - 11s 674us/step - loss: 0.6911 - acc: 0.5287 - val_loss: 0.6941 - val_acc: 0.5008
Epoch 3/50
15658/15658 [==============================] - 10s 665us/step - loss: 0.6909 - acc: 0.5289 - val_loss: 0.6929 - val_acc: 0.5005
Epoch 4/50
15658/15658 [==============================] - 10s 662us/step - loss: 0.6904 - acc: 0.5287 - val_loss: 0.6914 - val_acc: 0.5000
Epoch 5/50
15658/15658 [==============================] - 10s 659us/step - loss: 0.6895 - acc: 0.5291 - val_loss: 0.6896 - val_acc: 0.4998
Epoch 6/50
15658/15658 [==============================] - 10s 650us/step - loss: 0.6879 - acc: 0.5284 - val_loss: 0.6855 - val_acc:

In [72]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    model_input = Input(shape=img_dim)
    
#     for i in range(1):
    inputs = Lambda(cut, arguments={'index':0})(model_input)
    print inputs
    print model_input
    
    cnn1 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
    cnn2 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
    pool1 = MaxPooling1D(pool_size=(7))(cnn2)
    drop1 = Dropout(0.25)(pool1)

    cnn3 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
    cnn4 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
    pool2 = MaxPooling1D(pool_size=(6))(cnn4)
    drop2 = Dropout(0.25)(pool2)

    cnn5 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
    cnn6 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
    pool3 = MaxPooling1D(pool_size=(6))(cnn6)
    drop3 = Dropout(0.25)(pool3)
    
    flatten1 = Flatten()(drop3)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=25, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("lambda_1/strided_slice:0", shape=(?, 3640, 1), dtype=float32)
Tensor("input_67:0", shape=(?, 12, 3640, 1), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 40s 3ms/step - loss: 0.6926 - acc: 0.5158 - val_loss: 0.6943 - val_acc: 0.5010
Epoch 2/50
15658/15658 [==============================] - 10s 646us/step - loss: 0.6910 - acc: 0.5289 - val_loss: 0.6930 - val_acc: 0.5010
Epoch 3/50
15658/15658 [==============================] - 10s 651us/step - loss: 0.6899 - acc: 0.5293 - val_loss: 0.6910 - val_acc: 0.5010
Epoch 4/50
15658/15658 [==============================] - 10s 660us/step - loss: 0.6877 - acc: 0.5349 - val_loss: 0.6867 - val_acc: 0.5230
Epoch 5/50
15658/15658 [==============================] - 10s 644us/step - loss: 0.6831 - acc: 0.5570 - val_loss: 0.6825 - val_acc: 0.5573
Epoch 6/50
15658/15658 [==============================] - 10s 634us/step - loss: 0.6690 - acc: 0.6022 - val_loss: 0.6582 - val_a

In [18]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(ROWS):
        inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    flatten1 = Flatten()(concat)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_13/concat:0", shape=(?, 10, 60), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 63s 4ms/step - loss: 0.6231 - acc: 0.6334 - val_loss: 0.5033 - val_acc: 0.7688
Epoch 2/50
15658/15658 [==============================] - 37s 2ms/step - loss: 0.5270 - acc: 0.7360 - val_loss: 0.4671 - val_acc: 0.8203
Epoch 3/50
15658/15658 [==============================] - 37s 2ms/step - loss: 0.4951 - acc: 0.7599 - val_loss: 0.4753 - val_acc: 0.8367
Epoch 4/50
15658/15658 [==============================] - 37s 2ms/step - loss: 0.4789 - acc: 0.7682 - val_loss: 0.4689 - val_acc: 0.8227
Epoch 5/50
15658/15658 [==============================] - 37s 2ms/step - loss: 0.4644 - acc: 0.7786 - val_loss: 0.5169 - val_acc: 0.8590
Epoch 6/50
15658/15658 [==============================] - 38s 2ms/step - loss: 0.4506 - acc: 0.7865 - val_loss: 0.5555 - val_acc: 0.8707
Epoch 7/50
15658/15658 [==============================] - 37s

In [23]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(ROWS):
        inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(8))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(7))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(7))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    flatten1 = Flatten()(concat)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_18/concat:0", shape=(?, 6, 60), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.6035 - acc: 0.6653 - val_loss: 0.4721 - val_acc: 0.8290
Epoch 2/50
15658/15658 [==============================] - 38s 2ms/step - loss: 0.5068 - acc: 0.7574 - val_loss: 0.4005 - val_acc: 0.8345
Epoch 3/50
15658/15658 [==============================] - 39s 2ms/step - loss: 0.4567 - acc: 0.7889 - val_loss: 0.3656 - val_acc: 0.8768
Epoch 4/50
15658/15658 [==============================] - 39s 2ms/step - loss: 0.4243 - acc: 0.8105 - val_loss: 0.3522 - val_acc: 0.8800
Epoch 5/50
15658/15658 [==============================] - 39s 2ms/step - loss: 0.4065 - acc: 0.8203 - val_loss: 0.3052 - val_acc: 0.8908
Epoch 6/50
15658/15658 [==============================] - 39s 2ms/step - loss: 0.3949 - acc: 0.8289 - val_loss: 0.2905 - val_acc: 0.8915
Epoch 7/50
15658/15658 [==============================] - 39s 

In [30]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(ROWS):
        inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(inputs)
        cnn2 = Convolution1D(6, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(7, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(5, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=1)(feature_list)
    print concat
    
    lstm_out1 = LSTM(32,return_sequences=True)(concat)

    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 8
nb_epoch = 40
img_dim = (ROWS,COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12,3640,1)
xvalid = X_valid.reshape(-1,12,3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_25/concat:0", shape=(?, 120, 5), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 628s 40ms/step - loss: 0.5684 - acc: 0.6874 - val_loss: 0.4242 - val_acc: 0.8458
Epoch 2/50
15658/15658 [==============================] - 587s 37ms/step - loss: 0.4549 - acc: 0.7869 - val_loss: 0.4177 - val_acc: 0.8748
Epoch 3/50
15658/15658 [==============================] - 552s 35ms/step - loss: 0.4275 - acc: 0.8028 - val_loss: 0.4810 - val_acc: 0.8758
Epoch 4/50
15658/15658 [==============================] - 605s 39ms/step - loss: 0.4067 - acc: 0.8176 - val_loss: 0.4531 - val_acc: 0.8608
Epoch 5/50
15658/15658 [==============================] - 602s 38ms/step - loss: 0.3982 - acc: 0.8215 - val_loss: 0.3703 - val_acc: 0.8868
Epoch 6/50
15658/15658 [==============================] - 558s 36ms/step - loss: 0.3910 - acc: 0.8263 - val_loss: 0.3263 - val_acc: 0.8788
Epoch 7/50
15658/15658 [=========================

In [32]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def createDenseNet(nb_classes, img_dim):

    model_input = Input(shape=img_dim)

    cnn1 = Convolution1D(24, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
    cnn2 = Convolution1D(24, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
    pool1 = MaxPooling1D(pool_size=(7))(cnn2)
    drop1 = Dropout(0.25)(pool1)

    cnn3 = Convolution1D(28, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
    cnn4 = Convolution1D(28, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
    pool2 = MaxPooling1D(pool_size=(6))(cnn4)
    drop2 = Dropout(0.25)(pool2)

    cnn5 = Convolution1D(20, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
    cnn6 = Convolution1D(20, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
    pool3 = MaxPooling1D(pool_size=(6))(cnn6)
    drop3 = Dropout(0.25)(pool3)
    
    flatten1 = Flatten()(drop3)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)
# densenet_depth = 46
# densenet_growth_rate = 8

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 79s 5ms/step - loss: 0.5721 - acc: 0.6871 - val_loss: 0.5640 - val_acc: 0.8340
Epoch 2/50
15658/15658 [==============================] - 39s 3ms/step - loss: 0.4426 - acc: 0.8010 - val_loss: 0.4232 - val_acc: 0.8772
Epoch 3/50
15658/15658 [==============================] - 40s 3ms/step - loss: 0.4058 - acc: 0.8215 - val_loss: 0.4238 - val_acc: 0.8852
Epoch 4/50
15658/15658 [==============================] - 40s 3ms/step - loss: 0.3766 - acc: 0.8358 - val_loss: 0.3791 - val_acc: 0.8878
Epoch 5/50
15658/15658 [==============================] - 40s 3ms/step - loss: 0.3639 - acc: 0.8422 - val_loss: 0.3318 - val_acc: 0.8960
Epoch 6/50
15658/15658 [==============================] - 40s 3ms/step - loss: 0.3535 - acc: 0.8479 - val_loss: 0.3634 - val_acc: 0.8910
Epoch 7/50
15658/15658 [==============================] - 40s 3ms/step - loss: 0.3458 - acc: 0.8532 - val_loss: 0.3292 - val_acc:

In [33]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def createDenseNet(nb_classes, img_dim):

    model_input = Input(shape=img_dim)

    cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
    cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
    pool1 = MaxPooling1D(pool_size=(7))(cnn2)
    drop1 = Dropout(0.25)(pool1)

    cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
    cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
    pool2 = MaxPooling1D(pool_size=(6))(cnn4)
    drop2 = Dropout(0.25)(pool2)

    cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
    cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
    pool3 = MaxPooling1D(pool_size=(6))(cnn6)
    drop3 = Dropout(0.25)(pool3)
    
    flatten1 = Flatten()(drop3)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)
# densenet_depth = 46
# densenet_growth_rate = 8

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 114s 7ms/step - loss: 0.5684 - acc: 0.6943 - val_loss: 0.4515 - val_acc: 0.8285
Epoch 2/50
15658/15658 [==============================] - 73s 5ms/step - loss: 0.4610 - acc: 0.7850 - val_loss: 0.4700 - val_acc: 0.8582
Epoch 3/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.4138 - acc: 0.8157 - val_loss: 0.4492 - val_acc: 0.8838
Epoch 4/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.3878 - acc: 0.8304 - val_loss: 0.3525 - val_acc: 0.8915
Epoch 5/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.3687 - acc: 0.8396 - val_loss: 0.3950 - val_acc: 0.8978
Epoch 6/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.3571 - acc: 0.8467 - val_loss: 0.2954 - val_acc: 0.9040
Epoch 7/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.3469 - acc: 0.8518 - val_loss: 0.3125 - val_acc

In [40]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def createDenseNet(nb_classes, img_dim):

    model_input = Input(shape=img_dim)

    cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
    cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
    pool1 = MaxPooling1D(pool_size=(7))(cnn2)
    drop1 = Dropout(0.25)(pool1)

    cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
    cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
    pool2 = MaxPooling1D(pool_size=(6))(cnn4)
    drop2 = Dropout(0.25)(pool2)

    cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
    cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
    pool3 = MaxPooling1D(pool_size=(6))(cnn6)
    drop3 = Dropout(0.25)(pool3)
    
    flatten1 = Flatten()(drop3)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)
# densenet_depth = 46
# densenet_growth_rate = 8

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 118s 8ms/step - loss: 0.6048 - acc: 0.6558 - val_loss: 0.4894 - val_acc: 0.8240
Epoch 2/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.4799 - acc: 0.7718 - val_loss: 0.4241 - val_acc: 0.8622
Epoch 3/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.4371 - acc: 0.7993 - val_loss: 0.3705 - val_acc: 0.8728
Epoch 4/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.4065 - acc: 0.8202 - val_loss: 0.4089 - val_acc: 0.8790
Epoch 5/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.3865 - acc: 0.8313 - val_loss: 0.4643 - val_acc: 0.8665
Epoch 6/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.3690 - acc: 0.8382 - val_loss: 0.3088 - val_acc: 0.8835
Epoch 7/50
15658/15658 [==============================] - 72s 5ms/step - loss: 0.3570 - acc: 0.8459 - val_loss: 0.3499 - val_acc

In [43]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def createDenseNet(nb_classes, img_dim):

    model_input = Input(shape=img_dim)

    cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
    cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
    pool1 = MaxPooling1D(pool_size=(7))(cnn2)
    drop1 = Dropout(0.25)(pool1)

    cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
    cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
    pool2 = MaxPooling1D(pool_size=(6))(cnn4)
    drop2 = Dropout(0.25)(pool2)

    cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
    cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
    pool3 = MaxPooling1D(pool_size=(6))(cnn6)
    drop3 = Dropout(0.25)(pool3)
    
    drop3 = LSTM(64,return_sequences=True)(drop3)
    
    flatten1 = Flatten()(drop3)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 64
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)
# densenet_depth = 46
# densenet_growth_rate = 8

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 191s 12ms/step - loss: 0.6046 - acc: 0.6434 - val_loss: 0.4563 - val_acc: 0.8183
Epoch 2/50
15658/15658 [==============================] - 141s 9ms/step - loss: 0.4709 - acc: 0.7755 - val_loss: 0.4471 - val_acc: 0.8675
Epoch 3/50
15658/15658 [==============================] - 141s 9ms/step - loss: 0.4230 - acc: 0.8092 - val_loss: 0.4463 - val_acc: 0.8848
Epoch 4/50
15658/15658 [==============================] - 142s 9ms/step - loss: 0.3898 - acc: 0.8262 - val_loss: 0.4102 - val_acc: 0.8880
Epoch 5/50
15658/15658 [==============================] - 142s 9ms/step - loss: 0.3709 - acc: 0.8387 - val_loss: 0.4229 - val_acc: 0.8955
Epoch 6/50
15658/15658 [==============================] - 143s 9ms/step - loss: 0.3562 - acc: 0.8497 - val_loss: 0.4178 - val_acc: 0.8990
Epoch 7/50
15658/15658 [==============================] - 143s 9ms/step - loss: 0.3496 - acc: 0.8501 - val_loss: 0.3685 - 

In [46]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def createDenseNet(nb_classes, img_dim):

    model_input = Input(shape=img_dim)

    cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
    cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
    pool1 = MaxPooling1D(pool_size=(7))(cnn2)
    drop1 = Dropout(0.25)(pool1)

    cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
    cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
    pool2 = MaxPooling1D(pool_size=(6))(cnn4)
    drop2 = Dropout(0.25)(pool2)

    cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
    cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                  activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
    pool3 = MaxPooling1D(pool_size=(6))(cnn6)
    drop3 = Dropout(0.25)(pool3)
    
    drop3 = LSTM(64,return_sequences=True)(drop3)
    
    flatten1 = Flatten()(drop3)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 64
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)
# densenet_depth = 46
# densenet_growth_rate = 8

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 191s 12ms/step - loss: 0.6176 - acc: 0.6370 - val_loss: 0.5732 - val_acc: 0.7113
Epoch 2/50
15658/15658 [==============================] - 141s 9ms/step - loss: 0.4820 - acc: 0.7673 - val_loss: 0.5084 - val_acc: 0.8645
Epoch 3/50
15658/15658 [==============================] - 143s 9ms/step - loss: 0.4301 - acc: 0.8043 - val_loss: 0.4753 - val_acc: 0.8760
Epoch 4/50
15658/15658 [==============================] - 143s 9ms/step - loss: 0.3989 - acc: 0.8219 - val_loss: 0.4674 - val_acc: 0.8877
Epoch 5/50
15658/15658 [==============================] - 143s 9ms/step - loss: 0.3774 - acc: 0.8332 - val_loss: 0.4233 - val_acc: 0.8930
Epoch 6/50
15658/15658 [==============================] - 143s 9ms/step - loss: 0.3635 - acc: 0.8432 - val_loss: 0.4139 - val_acc: 0.8962
Epoch 7/50
15658/15658 [==============================] - 143s 9ms/step - loss: 0.3533 - acc: 0.8474 - val_loss: 0.4063 - 

In [48]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=1)(feature_list)
    print concat
    
    lstm_out1 = LSTM(64,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_26/concat:0", shape=(?, 338, 30), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 594s 38ms/step - loss: 0.6003 - acc: 0.6506 - val_loss: 0.5385 - val_acc: 0.7800
Epoch 2/50
15658/15658 [==============================] - 522s 33ms/step - loss: 0.4698 - acc: 0.7766 - val_loss: 0.5017 - val_acc: 0.8340
Epoch 3/50
15658/15658 [==============================] - 496s 32ms/step - loss: 0.4319 - acc: 0.8027 - val_loss: 0.4476 - val_acc: 0.8568
Epoch 4/50
15658/15658 [==============================] - 471s 30ms/step - loss: 0.4037 - acc: 0.8186 - val_loss: 0.3745 - val_acc: 0.8808
Epoch 5/50
15658/15658 [==============================] - 469s 30ms/step - loss: 0.3739 - acc: 0.8379 - val_loss: 0.4018 - val_acc: 0.8873
Epoch 6/50
15658/15658 [==============================] - 471s 30ms/step - loss: 0.3607 - acc: 0.8463 - val_loss: 0.3288 - val_acc: 0.9005
Epoch 7/50
15658/15658 [========================

In [52]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

class Attention_layer(Layer): 
    def __init__(self,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')
 
        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)
 
        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)
 
        self.bias = bias
        super(Attention_layer, self).__init__(**kwargs)
 
    def build(self, input_shape):
        assert len(input_shape) == 3
 
        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
 
        super(Attention_layer, self).build(input_shape)
 
    def compute_mask(self, input, input_mask=None):
        return None
 
    def call(self, x, mask=None):
        uit = K.dot(x, self.W)
        if self.bias:
            uit += self.b
        uit = K.tanh(uit)
        a = K.exp(uit)
        if mask is not None:
            a *= K.cast(mask, K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())
        print a
        print x
        weighted_input = x * a
        print weighted_input
        return K.sum(weighted_input, axis=1)
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(2):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
#     lstm_out1 = LSTM(64,return_sequences=True)(concat)
#     print lstm_out1
    
    flatten1 = Attention_layer()(concat)

#     flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_30/concat:0", shape=(?, 169, 60), dtype=float32)
Tensor("attention_layer_1/div:0", shape=(?, 169, 60), dtype=float32)
Tensor("concatenate_30/concat:0", shape=(?, 169, 60), dtype=float32)
Tensor("attention_layer_1/mul:0", shape=(?, 169, 60), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 211s 13ms/step - loss: 0.6704 - acc: 0.5741 - val_loss: 0.6437 - val_acc: 0.6805
Epoch 2/50
15658/15658 [==============================] - 139s 9ms/step - loss: 0.5703 - acc: 0.6971 - val_loss: 0.5325 - val_acc: 0.8193
Epoch 3/50
15658/15658 [==============================] - 139s 9ms/step - loss: 0.4959 - acc: 0.7629 - val_loss: 0.5549 - val_acc: 0.8345
Epoch 4/50
15658/15658 [==============================] - 139s 9ms/step - loss: 0.4670 - acc: 0.7795 - val_loss: 0.4517 - val_acc: 0.8660
Epoch 5/50
15658/15658 [==============================] - 139s 9ms/step - loss: 0.4352 - acc: 0.8033 - val_loss: 0.4350 - 

In [54]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(3):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
#         print drop3
        feature_list.append(drop3)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(128,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_32/concat:0", shape=(?, 169, 90), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 481s 31ms/step - loss: 0.6135 - acc: 0.6412 - val_loss: 0.4682 - val_acc: 0.8210
Epoch 2/50
15658/15658 [==============================] - 403s 26ms/step - loss: 0.4820 - acc: 0.7650 - val_loss: 0.4769 - val_acc: 0.8640
Epoch 3/50
15658/15658 [==============================] - 405s 26ms/step - loss: 0.4437 - acc: 0.7942 - val_loss: 0.4975 - val_acc: 0.8770
Epoch 4/50
15658/15658 [==============================] - 403s 26ms/step - loss: 0.4200 - acc: 0.8092 - val_loss: 0.4809 - val_acc: 0.8810
Epoch 5/50
15658/15658 [==============================] - 405s 26ms/step - loss: 0.3942 - acc: 0.8249 - val_loss: 0.4326 - val_acc: 0.8975
Epoch 6/50
15658/15658 [==============================] - 410s 26ms/step - loss: 0.3701 - acc: 0.8398 - val_loss: 0.3873 - val_acc: 0.8998
Epoch 7/50
15658/15658 [========================

In [56]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(3):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
        cnn7 = Convolution1D(30, (3), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop3)
        cnn8 = Convolution1D(30, (3), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn7)
        pool4 = MaxPooling1D(pool_size=(3))(cnn8)
        drop4 = Dropout(0.25)(pool4)
        
#         print drop3
        feature_list.append(drop4)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(128,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.005, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=10, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=50,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_34/concat:0", shape=(?, 55, 90), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/50
15658/15658 [==============================] - 372s 24ms/step - loss: 0.6602 - acc: 0.5865 - val_loss: 0.5372 - val_acc: 0.7480
Epoch 2/50
15658/15658 [==============================] - 281s 18ms/step - loss: 0.5079 - acc: 0.7470 - val_loss: 0.4277 - val_acc: 0.8320
Epoch 3/50
15658/15658 [==============================] - 280s 18ms/step - loss: 0.3928 - acc: 0.8258 - val_loss: 0.3362 - val_acc: 0.8890
Epoch 4/50
15658/15658 [==============================] - 280s 18ms/step - loss: 0.3633 - acc: 0.8439 - val_loss: 0.3434 - val_acc: 0.8960
Epoch 5/50
15658/15658 [==============================] - 280s 18ms/step - loss: 0.3501 - acc: 0.8511 - val_loss: 0.2985 - val_acc: 0.8928
Epoch 6/50
15658/15658 [==============================] - 270s 17ms/step - loss: 0.3457 - acc: 0.8543 - val_loss: 0.3438 - val_acc: 0.8995
Epoch 7/50
15658/15658 [=========================

In [60]:
from keras.regularizers import l2
from keras.layers import *
from keras.models import *
from keras.utils import np_utils

def cut(x, index):
    x = x[:,index]
    return x

def createDenseNet(nb_classes, img_dim):

    feature_list = []
    
    model_input = Input(shape=img_dim)
    
    for i in range(3):
#         inputs = Lambda(cut, arguments={'index':i})(model_input)
#         print inputs
#         print model_input

        cnn1 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(model_input)
        cnn2 = Convolution1D(36, (21), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn1)
        pool1 = MaxPooling1D(pool_size=(7))(cnn2)
        drop1 = Dropout(0.25)(pool1)

#         drop1 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop1)
        
        cnn3 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop1)
        cnn4 = Convolution1D(42, (13), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn3)
        pool2 = MaxPooling1D(pool_size=(6))(cnn4)
        drop2 = Dropout(0.25)(pool2)
        
#         drop2 = BatchNormalization(axis=1, gamma_regularizer=l2(1E-4),beta_regularizer=l2(1E-4))(drop2)
        
        cnn5 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop2)
        cnn6 = Convolution1D(30, (9), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn5)
        pool3 = MaxPooling1D(pool_size=(6))(cnn6)
        drop3 = Dropout(0.25)(pool3)
        
        cnn7 = Convolution1D(30, (3), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(drop3)
        cnn8 = Convolution1D(30, (3), kernel_initializer='glorot_uniform', 
                      activation=keras.layers.advanced_activations.LeakyReLU(alpha=0.2))(cnn7)
        pool4 = MaxPooling1D(pool_size=(3))(cnn8)
        drop4 = Dropout(0.25)(pool4)
        
#         print drop3
        feature_list.append(drop4)
        
    concat = Concatenate(axis=2)(feature_list)
    print concat
    
    lstm_out1 = LSTM(128,return_sequences=True)(concat)
    
#     flatten1 = Attention_layer(lstm_out1)
    
    flatten1 = Flatten()(lstm_out1)
    
    x = Dense(nb_classes, kernel_initializer='glorot_uniform', activation='softmax')(flatten1)

    densenet = Model(inputs=model_input, outputs=x)

    return densenet

#define DenseNet parms
ROWS = 12
COLS = 3640
CHANNELS = 1
nb_classes = 2
batch_size = 32
nb_epoch = 40
img_dim = (ROWS*COLS,CHANNELS)

model = createDenseNet(nb_classes=nb_classes,img_dim=img_dim)

sgd = SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


earlystop = keras.callbacks.EarlyStopping(monitor='val_acc', patience=20, verbose=1, mode='auto')
xtrain = X_train.reshape(-1,12*3640,1)
xvalid = X_valid.reshape(-1,12*3640,1)
model.fit(xtrain, y_train, batch_size=batch_size, epochs=200,
         validation_data=(xvalid,y_valid),
         callbacks = [earlystop])

Tensor("concatenate_38/concat:0", shape=(?, 55, 90), dtype=float32)
Train on 15658 samples, validate on 4000 samples
Epoch 1/200
15658/15658 [==============================] - 352s 23ms/step - loss: 0.6843 - acc: 0.5297 - val_loss: 0.6778 - val_acc: 0.6278
Epoch 2/200
15658/15658 [==============================] - 269s 17ms/step - loss: 0.6600 - acc: 0.5924 - val_loss: 0.6352 - val_acc: 0.6478
Epoch 3/200
15658/15658 [==============================] - 270s 17ms/step - loss: 0.6354 - acc: 0.6246 - val_loss: 0.6029 - val_acc: 0.6720
Epoch 4/200
15658/15658 [==============================] - 268s 17ms/step - loss: 0.5999 - acc: 0.6639 - val_loss: 0.5167 - val_acc: 0.7732
Epoch 5/200
15658/15658 [==============================] - 269s 17ms/step - loss: 0.5510 - acc: 0.7099 - val_loss: 0.4929 - val_acc: 0.7632
Epoch 6/200
15658/15658 [==============================] - 269s 17ms/step - loss: 0.5042 - acc: 0.7475 - val_loss: 0.4498 - val_acc: 0.8242
Epoch 7/200
15658/15658 [==================